In [18]:
import pickle

single_run = None
with open('data/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('data/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)

with open('data/n_run1.pkl', 'rb') as rf:
    n_runs1 = pickle.load(rf)

In [2]:
import numpy as np
import time
from tabulate import tabulate


In [3]:
chromo_set_1, score_set_1 = single_run

In [26]:
n_results = np.array(n_runs)
n_results1 = np.array(n_runs1)

/var/folders/ms/33mxkwtn255b353qtr2c5m4r0000gn/T/ipykernel_80341/1858558450.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  n_results = np.array(n_runs)
/var/folders/ms/33mxkwtn255b353qtr2c5m4r0000gn/T/ipykernel_80341/1858558450.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  n_results1 = np.array(n_runs1)


In [7]:
from n_run_evolutions import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score, all_models_score_table

In [8]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [9]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

# Collect Common indexes from res of single run evolution


In [11]:
all_models_score_table.iloc[0, 1:]

Accuracy               0.8
Exec_Time_secs    0.736735
Name: 0, dtype: object

In [43]:
def pick_top_n_genes_n_run(genes, chromo_set, n=1):
    common_index = dict()
    chromo_len_in_each_gen = []
    for chromo in chromo_set:
        if n==1:
            chromo_len_in_each_gen.append(np.count_nonzero(chromo == 1))
            for i in range(len(chromo)):
                if chromo[i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1
        else:
            best_gen = np.array(chromo[:, -1])
            chromo_len_in_each_gen.append(np.count_nonzero(best_gen[0] == 1))
            for i in range(len(best_gen[0])):
                if best_gen[0][i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1

    if genes == -1:
        genes = len(common_index)

    sorted_common_index = np.array(sorted(
        common_index.items(), key=lambda x: x[1], reverse=True)[:genes])
    return sorted_common_index[:, 0], len(sorted_common_index), np.array(chromo_len_in_each_gen)

In [13]:
X_train.shape

(750, 1553)

# Collect Common indexes from res of n run evolution


In [35]:
n_results1[:, :, -1]

array([[array([0, 1, 0, ..., 1, 0, 0]), 0.764],
       [array([1, 0, 0, ..., 0, 0, 0]), 0.804],
       [array([0, 0, 0, ..., 1, 0, 0]), 0.804],
       [array([0, 1, 0, ..., 1, 1, 0]), 0.812],
       [array([0, 1, 0, ..., 1, 0, 0]), 0.78],
       [array([1, 1, 1, ..., 1, 0, 1]), 0.776],
       [array([0, 0, 0, ..., 1, 1, 1]), 0.748],
       [array([1, 0, 0, ..., 1, 0, 0]), 0.772],
       [array([1, 1, 0, ..., 0, 1, 0]), 0.788],
       [array([0, 0, 0, ..., 1, 0, 1]), 0.816],
       [array([0, 1, 0, ..., 0, 0, 0]), 0.776],
       [array([0, 0, 0, ..., 0, 0, 0]), 0.784],
       [array([1, 0, 1, ..., 0, 1, 0]), 0.78],
       [array([0, 0, 1, ..., 0, 0, 1]), 0.78],
       [array([0, 0, 0, ..., 1, 1, 1]), 0.748],
       [array([0, 0, 1, ..., 0, 0, 0]), 0.804],
       [array([1, 1, 1, ..., 0, 1, 0]), 0.812],
       [array([0, 1, 0, ..., 0, 0, 0]), 0.76],
       [array([0, 0, 1, ..., 1, 1, 1]), 0.784],
       [array([1, 1, 0, ..., 1, 0, 0]), 0.768],
       [array([1, 0, 1, ..., 0, 0, 0]), 0.79

# Final Results

In [40]:
def tabulate_results(chromo_set, n):
    genes = [100, 150, 200, 250, -1]
    accuracy = []
    time_to_train = []
    sentiment_ratio = []
    for i in genes:
        top_n_genes, common_count, chromo_len = pick_top_n_genes_n_run(i, chromo_set, n)
        acc, ttt, sr = test_accuracy(top_n_genes)
        accuracy.append(acc)
        time_to_train.append(ttt)
        sentiment_ratio.append(sr)

    
    table_data = {'Genes': genes,
                'Accuracy': accuracy,
                'Time Taken to Train': time_to_train,
                'Sentiment Ratio(p/n)': sentiment_ratio
                }

    print('Actual chromosome length in X_Test: ', X_test.shape[1])
    print('RandomForest with full length -', all_models_score_table.iloc[0, 1:])
    print('Total Common genes count ~(-1): ', common_count)
    print(chromo_len)
    print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))


Comparitive results for Single run evolution

In [41]:
tabulate_results(chromo_set_1, 1)

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    0.736735
Name: 0, dtype: object
Total Common genes count ~(-1):  1362
[100 100 318 318 318 318 318 296 296 296 296 458 495 495 495 495 447 407
 407 458 458 458 466 466 580 382 382 382 578 578]
┌─────────┬────────────┬───────────────────────┬────────────────────────┐
│   Genes │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     100 │      0.688 │              0.62119  │ (30, 70)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     150 │      0.72  │              0.302092 │ (35, 64)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     200 │      0.716 │              0.296814 │ (36, 64)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     250 │      0.724

Comparitive results for 30 run evolution

In [44]:
tabulate_results(n_results, 30)

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    0.736735
Name: 0, dtype: object
Total Common genes count ~(-1):  1553
[612 506 369 542 613 643 510 580 436 415 535 476 585 422 434 448 590 536
 468 596 438 493 538 450 463 606 468 477 498 528]
┌─────────┬────────────┬───────────────────────┬────────────────────────┐
│   Genes │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     100 │      0.796 │              0.503496 │ (37, 62)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     150 │      0.808 │              0.268733 │ (38, 61)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     200 │      0.836 │              0.29827  │ (41, 58)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     250 │      0.828

In [46]:
tabulate_results(n_results1, 30)

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    0.736735
Name: 0, dtype: object
Total Common genes count ~(-1):  1553
[451 478 568 550 480 512 455 596 492 511 248 502 392 530 441 574 576 503
 683 552 575 265 376 571 635 650 548 404 553 319]
┌─────────┬────────────┬───────────────────────┬────────────────────────┐
│   Genes │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     100 │      0.78  │              0.583154 │ (40, 59)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     150 │      0.788 │              0.292918 │ (42, 57)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     200 │      0.788 │              0.292467 │ (42, 57)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     250 │      0.788